In [ ]:
from uuid import uuid4
import numpy as np
import random

In [ ]:
class Edge:
    def __init__(self, id: str, source_node, target_node, weight: int = None, is_directed: bool = False, initial_attrs: dict = {}):
        self.id = id
        self.source_node = source_node
        self.target_node = target_node
        self.is_directed = is_directed
        self.weight = weight if weight is not None else np.random.randint(-100, 100)
        self.attrs = {
            **initial_attrs
        }

In [ ]:
from copy import deepcopy


class Node:
    def __init__(self, id: str, edges: dict[str, Edge] = {}, neighbors: dict[str, 'Node'] = {}, coords: tuple[int, int] = None, initial_attrs: dict = {}):
        self.id = id
        self.edges = edges # Lista de Aristas conectadas directamente
        self.neighbors = neighbors # Lista de Nodos directamente conectados
        self.coords = coords if coords else (round(np.random.random() * 100, 4), round(np.random.random() * 100, 4))
        self.attrs = {
            **initial_attrs
        }
    
    def add_connection(self, edge_name: str, target_node: 'Node', weight: int = None, is_directed: bool = False, edge_attrs: dict = {}):
        existing_edge = self.edges.get(edge_name)
        if existing_edge:
            #print("La arista {} ya existe dentro del nodo {}".format(existing_edge.id, self.id))
            existing_edge.target_node = target_node
            existing_edge.weight = weight
            existing_edge.is_directed = is_directed
            existing_edge.attrs.update(edge_attrs)
        else:
            #print("La arista {} NO existe dentro del nodo {}".format(edge_name, self.id))
            existing_edge = Edge(edge_name, self, target_node, weight, is_directed, edge_attrs)
        
        self.edges[edge_name] = existing_edge
        #not self.neighbors.get(target_node.id) and 
        if self.id != target_node.id:
            # Si el nodo destino no esta agregado a la lista de vecinos, agregarlo
            #print("Adding neighbor {} to node {}".format(target_node.id, self.id))
            self.neighbors[target_node.id] = target_node
        #print("Node {} with DEGREE {}".format(self.id, self.get_degree()))
        #print(self.neighbors)
        #print(self.edges)
        #print("="*30)
        return existing_edge

    #def correct_the_fuck(self):
        
    #    edges_names = list(self.edges.keys())
    #    for edg in edges_names:
    #        if not edg.startswith(self.id):
    #            del self.edges[edg]
        
        
    #    print(self.neighbors)
    #    print(self.edges)
        

    def get_degree(self):
        return len([ x for x in self.edges if self.edges[x].source_node.id == self.id ])

In [ ]:
class Graph:
    
    def __init__(self, graph_name):
        self.name = graph_name
        self.nodes = {}
        self.edges = {}
        self.attrs = {}

    def add_node(self, node: Node):
        existing_node = node
        self.nodes[node.id] = node
        # Agregar todos los nodos conectados a dicho nodo y aristas
        for nh in node.neighbors:
            self.nodes[nh] = node.neighbors.get(nh)
        for edg in node.edges:
            self.edges[edg] = node.edges.get(edg)
        return existing_node
    
    def get_random_node(self):
        node_names = list(self.nodes.keys())
        return self.nodes.get(random.choice(node_names))
    
    def get_graph_degree(self):
        return len(list(self.edges.keys()))
    
    def generate_graphviz_graph(self):
        graph_str = r"{} {}".format(
            "graph" if not self.is_directed else "digraph",
            self.graph_name
        )
        
        graph_str += "{\n"
        # Modificar para utilizar los nodos en vez de las aristas
        for edge in self.edges.values():
            src_node: Node = edge.source_node
            tgt_node: Node = edge.target_node
            graph_str += "{} -{} {};\n".format(
                src_node.id, ">" if self.is_directed else "-", tgt_node.id
            )
        
        graph_str += "}\n"
        
        return graph_str

    def export_to_graphviz_file(self, filename: str):
        graph_repr = self.generate_graphviz_graph()
        with open(filename, "w", encoding="utf-8") as gh:
            gh.write(graph_repr)
    

In [ ]:
def add_nodes_connection(n1, n1_edge_name, n2, n2_edge_name):
    pass

In [ ]:
def create_grid_graph(graph_name, m, n = None, use_diagonals = False):
    # Limpieza de datos
    grid_graph = Graph(graph_name)
    # m y n >= 2
    # si n es indefinido o invalido , n = m
    if n is None or n < 2:
        n = m    
    m = max(2, m)
    n = max(2, n)
    for i in range(m):
        for j in range(n):
            node_name = "GRID_GRAPH_{}".format(str(i * n + j))
            if j < n - 1:
                existing_node = grid_graph.nodes.get(node_name)
                if not existing_node:
                    existing_node = Node(node_name)
                    existing_node.coords = np.array([float(i), float(j)])
                next_node_name = "GRID_GRAPH_{}".format(str( i * n + j + 1))
                n1_edge_name = "{}-->{}".format(node_name, next_node_name)
                n2_edge_name = "{}-->{}".format(next_node_name, node_name)
                print("Connecting node {} to {} with {}".format(node_name, next_node_name, n1_edge_name))
                next_node = grid_graph.nodes.get(next_node_name)
                if not next_node:
                    next_node = Node(next_node_name)
                # Modificacion del nodo fuente
                existing_node.add_connection(n1_edge_name, next_node)
                # Modificacion del nodo destino
                next_node.add_connection(n2_edge_name, existing_node, existing_node.edges.get(n1_edge_name).weight)
                # Modificacion del grafo base
                grid_graph.add_node(existing_node)
                grid_graph.add_node(next_node)
            if i < m - 1:
                existing_node = grid_graph.nodes.get(node_name)
                if not existing_node:
                    existing_node = Node(node_name)
                    existing_node.coords = np.array([float(i), float(j)])
                next_node_name = "GRID_GRAPH_{}".format(str((i + 1) * n + j))
                n1_edge_name = "{}-->{}".format(node_name, next_node_name)
                n2_edge_name = "{}-->{}".format(next_node_name, node_name)
                print("Connecting node {} to {} with {}".format(node_name, next_node_name, n1_edge_name))
                next_node = grid_graph.nodes.get(next_node_name)
                if not next_node:
                    next_node = Node(next_node_name)
                # Modificacion del nodo fuente
                existing_node.add_connection(n1_edge_name, next_node)
                # Modificacion del nodo destino
                next_node.add_connection(n2_edge_name, existing_node, existing_node.edges.get(n1_edge_name).weight)
                # Modificacion del grafo base
                grid_graph.add_node(existing_node)
                grid_graph.add_node(next_node)
            if i < m - 1 and j < n - 1 and use_diagonals:
                existing_node = grid_graph.nodes.get(node_name)
                if not existing_node:
                    existing_node = Node(node_name)
                    existing_node.coords = np.array([float(i), float(j)])
                next_node_name = "GRID_GRAPH_{}".format(str((i + 1) * n + j + 1))
                n1_edge_name = "{}-->{}".format(node_name, next_node_name)
                n2_edge_name = "{}-->{}".format(next_node_name, node_name)
                print("Connecting node {} to {} with {}".format(node_name, next_node_name, n1_edge_name))
                next_node = grid_graph.nodes.get(next_node_name)
                if not next_node:
                    next_node = Node(next_node_name)
                # Modificacion del nodo fuente
                existing_node.add_connection(n1_edge_name, next_node)
                # Modificacion del nodo destino
                next_node.add_connection(n2_edge_name, existing_node, existing_node.edges.get(n1_edge_name).weight)
                # Modificacion del grafo base
                grid_graph.add_node(existing_node)
                grid_graph.add_node(next_node)
            if i > 0 and j < n - 1 and use_diagonals:
                existing_node = grid_graph.nodes.get(node_name)
                if not existing_node:
                    existing_node = Node(node_name)
                    existing_node.coords = np.array([float(i), float(j)])
                next_node_name = "GRID_GRAPH_{}".format(str((i - 1) * n + j + 1))
                n1_edge_name = "{}-->{}".format(node_name, next_node_name)
                n2_edge_name = "{}-->{}".format(next_node_name, node_name)
                print("Connecting node {} to {} with {}".format(node_name, next_node_name, n1_edge_name))
                next_node = grid_graph.nodes.get(next_node_name)
                if not next_node:
                    next_node = Node(next_node_name)
                # Modificacion del nodo fuente
                existing_node.add_connection(n1_edge_name, next_node)
                # Modificacion del nodo destino
                next_node.add_connection(n2_edge_name, existing_node, existing_node.edges.get(n1_edge_name).weight)
                # Modificacion del grafo base
                grid_graph.add_node(existing_node)
                grid_graph.add_node(next_node)
            print("{} --> {}".format(existing_node.id, existing_node.get_degree()))
        
    return grid_graph  

In [ ]:
grid_graph = create_grid_graph("GRID_GRAPH_SMALL", 3,3)

In [ ]:
grid_graph.get_graph_degree()

In [ ]:
len(grid_graph.nodes)

In [ ]:
grid_graph.nodes

In [ ]:
grid_graph.nodes["GRID_GRAPH_0"].neighbors